In [98]:
### __ Numerical Integration __ ###

import numpy as np

        # use "ndarray" rather than "list"

# acc

def l(v):
    return np.linalg.norm(v)

def rot(v, theta):
    m = [ [np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)] ]
    return np.matmul(m,v)
    
def acc(r0, i, id):
    
    r1 = r0 - sun['r'][i,:] if id != 's' else np.zeros(2)
    r2 = r0 - earth['r'][i,:] if id != 'e' else np.zeros(2)
    r3 = r0 - jupiter['r'][i,:] if id != 'j' else np.zeros(2)

    a1 = np.zeros(2) if not r1.any() else -r1/np.linalg.norm(r1) * G*Ms/np.linalg.norm(r1)**2 
    a2 = np.zeros(2) if not r2.any() else -r2/np.linalg.norm(r2) * G*Me/np.linalg.norm(r2)**2
    a3 = np.zeros(2) if not r3.any() else -r3/np.linalg.norm(r3) * G*Mj/np.linalg.norm(r3)**2
    
    return a1+a2+a3

# derivitive & integration

def d(r, v, i, id):
    return { 'dr_dt':v , 'dv_dt':acc(r, i, id) } 

def RK4Solver(info, i):

    id = 'n'
    if info is sun: id = 's'
    if info is earth: id = 'e'
    if info is jupiter: id = 'j'
    
    r = info['r'][i,:]
    v = info['v'][i,:]
    
    k1 = d( r , v , i ,id )
    k2 = d( r + k1['dr_dt']*h/2 , v + k1['dv_dt']*h/2 , i , id )
    k3 = d( r + k2['dr_dt']*h/2 , v + k2['dv_dt']*h/2 , i , id )
    k4 = d( r + k3['dr_dt']*h , v + k3['dv_dt']*h , i , id )

    Kr_h = h * ( k1['dr_dt'] + 2*k2['dr_dt'] + 2*k3['dr_dt'] + k4['dr_dt'] )/6
    Kv_h = h * ( k1['dv_dt'] + 2*k2['dv_dt'] + 2*k3['dv_dt'] + k4['dv_dt'] )/6

    return np.array([ r+Kr_h , v+Kv_h ])


# Constants . __ { 1 = time unit (yr) , 1 = length unit (AU) , 1 = mass unit (MoE) }

T_tr = 1/(365*24*60*60.)            # sec to yr
R_tr = 1/(1.496e11)                 # m to AU
M_tr = 1/(6e24)                     # kg to Mass of Earth

G_si = 6.673e-11                    # SI Gravitational Constant
G = G_si*(R_tr**3)/(M_tr*T_tr**2)   # translated Gravitational Constant


# Variables 1 . __ { 1 = time unit (yr) } { h = time step (yr) , h*N = tf-ti }

ti = 0                        # initial time = 0
tf = 120                      # final time = 120 years
N = 100*tf                    # 100 points per year

t = np.linspace(ti,tf,N)      # time array from ti to tf with N points 
h = t[2]-t[1]                 


# Variables 2 . __ { 1 = time unit (yr) , 1 = length unit (AU) }

earth = { 'r':np.zeros([N,2]) , 'v':np.zeros([N,2]) }                 
jupiter = { 'r':np.zeros([N,2]) , 'v':np.zeros([N,2]) }                      
sun = { 'r':np.zeros([N,2]) , 'v':np.zeros([N,2]) }                      

Mag = 1000

Ms = 100*Mag                               # adjustable 
Me = 1*Mag                                 # adjustable
Mj = 60*Mag                                # adjustable

Mc = Me+Mj+Ms

re = np.array([0,1])                              # adjustable
rj = np.array([1,-1])                              # adjustable
rs = np.array([-1,-1])                              # adjustable

rc = (Me*re+Mj*rj+Ms*rs)/Mc

earth['r'][0,:] = re
jupiter['r'][0,:] = rj
sun['r'][0,:] = rs

ve = rot(re-rc,np.pi/2)/l(re-rc) * np.sqrt(l(acc(re,0,'e'))*l(re-rc))
vj = rot(rj-rc,np.pi/2)/l(rj-rc) * np.sqrt(l(acc(rj,0,'j'))*l(rj-rc))
vs = rot(rs-rc,np.pi/2)/l(rs-rc) * np.sqrt(l(acc(rs,0,'s'))*l(rs-rc))

earth['v'][0,:] = ve
jupiter['v'][0,:] = vj
sun['v'][0,:] = vs


# Main Loop . __ { i = time counter }

for i in range(N-1):

    [ earth['r'][i+1,:] , earth['v'][i+1,:] ] = RK4Solver(earth, i)   
    [ jupiter['r'][i+1,:] , jupiter['v'][i+1,:] ] = RK4Solver(jupiter, i)
    [ sun['r'][i+1,:] , sun['v'][i+1,:] ] = RK4Solver(sun, i)
    

In [101]:
### __ Plots & Animation __ ###    

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation

from IPython.display import HTML


def init():                               ## s,e,j ##
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    time_lbl.set_text('')
    
    return (line1,line2,line3,time_lbl)

def animate(i):                           ## s,e,j ##
    earth_trail = 40
    jupiter_trail = 200
    sun_trail = 300
    
    time_lbl.set_text('t = ' + str(round(t[i],1)) + ' yr')
    line1.set_data(earth['r'][i:max(1,i-earth_trail):-1,0], earth['r'][i:max(1,i-earth_trail):-1,1])
    line2.set_data(jupiter['r'][i:max(1,i-jupiter_trail):-1,0], jupiter['r'][i:max(1,i-jupiter_trail):-1,1])
    line3.set_data(sun['r'][i:max(1,i-sun_trail):-1,0], sun['r'][i:max(1,i-sun_trail):-1,1])
    
    return (line1,line2,line3,time_lbl)


# plot framework

fig, ax = plt.subplots()
#fig, ax = plt.subplots(figsize=(13,4.5))

ax.set_xlim((-5.2, 5.2))
ax.set_ylim((-5.2, 5.2))
ax.set_aspect('equal')
ax.axis('off')


# result settings                         ## s,e,j ##

line1, = ax.plot([], [], 'o-',color = '#e3dccb',markerfacecolor = '#0077BE', markevery=10000, lw=2)   # line for Earth

line2, = ax.plot([], [], 'o-',color = '#e3dccb',markerfacecolor = '#f66338', markevery=10000, markersize = 8, lw=2)   # line for Jupiter

line3, = ax.plot([],[],'o-',color = '#e3dccb', markerfacecolor = "#FDB813", markevery=10000, markersize = 9 ,lw=2)

time_lbl = ax.text(-9, -5.5, '')

# result animation                       ## test_x.x ##

#plt.show()
plt.close()

matplotlib.rcParams['animation.embed_limit'] = 2**128

anm = animation.FuncAnimation(fig, animate, init_func=init, frames=1000, interval=5, blit=True)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=120, metadata=dict(artist='Me'), bitrate=1800)
anm.save('test_1.4.1_result.mp4', writer=writer)

In [102]:
HTML('<video controls src="test_1.4.1_result.mp4" width=680 >')